
---

## 🔹 6. 📦 **Artifacts & Storage**

---

### 📌 **What It Does**

MLflow lets you **store and organize important files** — such as prompts, retriever configs, output chains — and **tag runs** to trace which model, tool, or agent version was used.

---

### 🚀 **Common Use in GenAI/Agentic AI**

| Scenario                          | MLflow’s Role                                                          |
| --------------------------------- | ---------------------------------------------------------------------- |
| Logging LangChain prompts/configs | Save `.json`, `.txt`, or `.yaml` files using `log_artifact()`          |
| Chaining across models            | Use `get_artifact_uri()` to pass files into downstream pipelines       |
| Tracking versions and experiments | Tag each run (e.g., `model=gpt-4o`, `agent=react`, `retriever=chroma`) |

---

### ⚙️ **Key Functions with Usage**

| Function                    | Description                                                             | Example                                                  |
| --------------------------- | ----------------------------------------------------------------------- | -------------------------------------------------------- |
| `mlflow.log_artifact()`     | Save a file or folder as part of the current run                        | `mlflow.log_artifact("prompt_template.txt")`             |
| `mlflow.set_tags()`         | Add searchable tags like model version, agent type, retriever used      | `mlflow.set_tags({"model": "gpt-4o", "agent": "react"})` |
| `mlflow.get_artifact_uri()` | Get URI to retrieve the logged file (e.g., for use in deployment later) | `mlflow.get_artifact_uri("prompt_template.txt")`         |


---

### 🧠 Tips for GenAI Projects

| Task                     | Recommended Practice                                              |
| ------------------------ | ----------------------------------------------------------------- |
| Storing multiple prompts | Create `prompts/` folder, then `mlflow.log_artifacts("prompts/")` |
| Debugging chained output | Save intermediate steps as `.json` using `log_artifact()`         |
| Cross-step reuse in DAGs | Use `get_artifact_uri()` in LangGraph to pass data between nodes  |
| Auto-tag agent runs      | Use `mlflow.set_tags()` with model, retriever, dataset names      |

---



In [ ]:

### ✅ Real-Time Example: Wrap LangChain Agent with Logging + Prompt Artifacts

import mlflow
import mlflow.pyfunc
from langchain.agents import initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

# 1. Define the PythonModel wrapper for your agent
class MyAgentWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        tools = load_tools(["serpapi", "llm-math"], llm=ChatOpenAI(model="gpt-4o"))
        self.agent = initialize_agent(tools, ChatOpenAI(), agent="zero-shot-react-description")

    def predict(self, context, inputs):
        query = inputs.iloc[0]  # assuming single-row input
        return self.agent.run(query)

# 2. Save prompt templates or logic as artifacts
with open("prompt_template.txt", "w") as f:
    f.write("Use the toolset wisely and answer: {query}")

# 3. Log everything with MLflow
with mlflow.start_run(run_name="agent-logging"):
    mlflow.pyfunc.log_model(
        artifact_path="agent_model",
        python_model=MyAgentWrapper()
    )
    mlflow.log_artifact("prompt_template.txt")
    mlflow.log_params({"agent_type": "react", "model": "gpt-4o"})
    mlflow.log_metrics({"tools_used": 2, "tool_success_rate": 0.98})


In [ ]:

### ✅ Real-Time Example: Log Prompt & Retrieve URI

import mlflow

# Prepare and save your chain/prompt/config
with open("prompt_template.txt", "w") as f:
    f.write("You are a helpful assistant. Answer the following:\n{question}")

with mlflow.start_run(run_name="artifact-storage"):

    # Log the prompt template file
    mlflow.log_artifact("prompt_template.txt")

    # Set meaningful tags for tracking
    mlflow.set_tags({
        "model": "gpt-4o",
        "retriever": "chroma",
        "agent": "react"
    })

    # Get and use the URI (can be passed to LangChain/LangGraph loaders)
    uri = mlflow.get_artifact_uri("prompt_template.txt")
    print("🔗 Artifact URI:", uri)
